# Benchmark After Feature Engineering
This notebook performs a comprehensive benchmark of machine learning models for predicting thermodynamic properties of molecules **after** applying feature engineering techniques. The goal is to evaluate the impact of additional molecular descriptors on model performance and compare results against the baseline established in the "Before Feature Engineering" benchmark.

## Workflow Summary
1. **Environment Setup**: Install RDKit for molecular descriptor calculation
2. **Data Loading & Feature Engineering**: Load dataset and compute additional molecular descriptors
3. **Feature Leakage Prevention**: Explicit exclusion of target variables from feature set
4. **Data Splitting**: Stratified train/test split based on molecular size
5. **Model Training & Evaluation**: Hyperparameter optimization and performance assessment

## Feature Engineering Strategy

### **Original Features**
- **Atomic Counts**: Individual atom counts (C, H, N, O, etc.)
- **Functional Groups**: Presence/absence of specific chemical groups
- **Basic Molecular Properties**: From quantum mechanical calculations

### **Engineered Features Added**
| **Feature Category** | **Description** | **Molecular Insight** |
|---------------------|-----------------|----------------------|
| **3D Inertia Descriptors** | `inertia_sum`, `inertia_ratio` | Molecular shape and size in 3D space |
| **Topological Polar Surface Area (TPSA)** | `TPSA` | Molecular polarity and membrane permeability |
| **Molar Refractivity (MR)** | `MR` | Molecular volume and polarizability |
| **Gasteiger Charge Statistics** | `chg_mean`, `chg_std` | Electronic charge distribution patterns |

### **Why These Descriptors?**
- **3D Inertia**: Captures molecular geometry effects on thermodynamic properties
- **TPSA**: Relates to intermolecular interactions and solvation effects
- **Molar Refractivity**: Connected to molecular polarizability and dispersion forces
- **Charge Statistics**: Reflects electronic distribution affecting chemical reactivity


**Regressor Selection Rationale** --> Same As "_Before Feature Engineering_" notebook!


## Feature Engineering Hypothesis
**Hypothesis**: Adding molecular descriptors that capture 3D geometry, electronic distribution, and physicochemical properties will improve model performance by:

1. **Better Representation**: More complete molecular characterization
2. **Reduced Bias**: Less reliance on simple atom counts
3. **Enhanced Interactions**: Capture synergistic effects between different molecular aspects
4. **Improved Generalization**: Better performance on diverse molecular structures

## Expected Outcomes
This benchmark will help us:
1. **Quantify Feature Engineering Impact**: Direct performance comparison with baseline
2. **Identify Most Benefited Properties**: Which targets gain most from additional features
3. **Model-Specific Insights**: How different algorithms leverage engineered features
4. **Guide Future Feature Engineering**: Inform selection of additional descriptors


In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 17.9 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem, rdMolDescriptors, Descriptors

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from scipy.stats import uniform, randint
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Helper functions
def compute_descriptors(smiles_series):
    """Compute 3D inertia, TPSA, MR, and Gasteiger charge stats."""
    cols = ('inertia_sum','inertia_ratio','TPSA','MR','chg_mean','chg_std')
    records = []
    for smi in smiles_series:
        m = Chem.AddHs(Chem.MolFromSmiles(smi))     # Convert SMILES to molecule
        if AllChem.EmbedMolecule(m, randomSeed=42) != 0:
            vals = [np.nan]*6                       # If embedding fails, return NaNs
        else:
            AllChem.UFFOptimizeMolecule(m, confId=0)    # Optimize the 3D structure
            I1 = rdMolDescriptors.CalcPMI1(m)       # Calculate principal moments of inertia
            I2 = rdMolDescriptors.CalcPMI2(m)       # Calculate second moment of inertia
            I3 = rdMolDescriptors.CalcPMI3(m)       # Calculate third moment of inertia
            vals = [                                # Sum of inertia values
                I1+I2+I3,
                I1/(I2+1e-6),       
                rdMolDescriptors.CalcTPSA(m),
                Descriptors.MolMR(m)
            ]
            AllChem.ComputeGasteigerCharges(m)      # Compute Gasteiger charges
            ch = [float(a.GetProp('_GasteigerCharge')) for a in m.GetAtoms()]   # Extract charges
            vals += [np.mean(ch), np.std(ch)]       # Mean and std of charges
        records.append(dict(zip(cols, vals)))       # Create a record for each SMILES
    return pd.DataFrame.from_records(records)

In [ ]:
# 3. Load & prepare dataset
df = pd.read_csv('dataset_9May.csv')
smiles = df.pop('Molecule')

# Compute and append descriptors
desc_df = compute_descriptors(smiles)
df = pd.concat([df, desc_df], axis=1)

# Define targets
targets = ['gap','mu','alpha','homo','lumo','r2','zpve','U0','U','H','G','Cv']

# Define features — **EXCLUDE all targets explicitly!**
features = [c for c in df.columns if c not in targets + ['Molecule']]

# Rebuild feature matrix without leakage
X = df[features].fillna(0)
Y = df[targets]

# Sanity check — should print empty set
print("Targets in features (should be empty):", set(targets).intersection(set(X.columns)))

# Train/test split stratified by molecule size
atoms = [c for c in df.columns if len(c)==1 and c.isupper()]
size = df[atoms].sum(axis=1)
X_tr, X_te, Y_tr, Y_te = train_test_split(
    X, Y, test_size=0.2, random_state=42,
    stratify=pd.qcut(size, 5)
)

# Log transform for mu and r2 - training and test
Y_tr[['mu', 'r2']] = np.log1p(Y_tr[['mu', 'r2']])

Targets in features (should be empty): set()


In [ ]:
# --- Define preprocessing pipeline
pre = Pipeline([
    ('scale', StandardScaler()),
    ('var', VarianceThreshold(1e-5))
])

# --- Define regressor search spaces
param_spaces = {
    'HGBR': {
        'model__max_iter': randint(300, 800),
        'model__learning_rate': uniform(0.01, 0.15),
        'model__max_depth': randint(3, 10),
        'model__l2_regularization': uniform(0, 0.5),
        'model__max_leaf_nodes': randint(20, 100)
    },
    'RF': {
        'model__n_estimators': [100, 300, 500],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_leaf': [1, 5, 10],
        'model__max_features': ['sqrt', 0.5, 0.8]
    },
    'Ridge': {
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
    }
}

# --- Define regressors
regressors = {
    'HGBR': HistGradientBoostingRegressor(random_state=42),
    'RF': RandomForestRegressor(random_state=42, n_jobs=-1),
    'Ridge': Ridge()
}

# --- Results container
results = {}

In [ ]:
# --- Loop over targets and regressors
for target in targets:
    print(f"\n🔍 Target: {target}")
    results[target] = {}
    for name, model in regressors.items():
        print(f"  ➤ Tuning {name}...")
        pipe = Pipeline([
            ('pre', pre),
            ('model', model)
        ])
        # Use RandomizedSearchCV for hyperparameter tuning with less computational cost
        tuned_model = RandomizedSearchCV(
            pipe, param_spaces[name],
            scoring='neg_root_mean_squared_error',
            cv=3,
            n_iter=20,
            n_jobs=-1,
            random_state=42
        )
        # Fit the model on training data
        tuned_model.fit(X_tr, Y_tr[target])
        # Get the best pipeline
        best_pipe = tuned_model.best_estimator_
        # Predict on test data
        preds_ = best_pipe.predict(X_te)

        # Inverse transform for log-transformed targets
        if target in ['mu', 'r2']:
            preds_ = np.expm1(preds_)
            true_vals = np.expm1(Y_te[target])
        else:
            true_vals = Y_te[target]

        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(Y_te[target], preds_))
        mae = mean_absolute_error(Y_te[target], preds_)
        r2 = r2_score(Y_te[target], preds_)

        results[target][name] = {
            'RMSE': rmse, 'MAE': mae, 'R2': r2
        }
        print(f"    ✓ {name}: RMSE={rmse:.4f}, MAE={mae:.4f}, R2={r2:.4f}")

# --- Convert to DataFrame
benchmark_df = pd.concat({
    t: pd.DataFrame(v).T for t, v in results.items()
}, axis=0)
benchmark_df.index.names = ['Target', 'Model']


🔍 Target: gap
  ➤ Tuning HGBR...
    ✓ HGBR: RMSE=0.0179, MAE=0.0125, R2=0.8519
  ➤ Tuning RF...
    ✓ RF: RMSE=0.0177, MAE=0.0122, R2=0.8543
  ➤ Tuning Ridge...
    ✓ Ridge: RMSE=0.0309, MAE=0.0252, R2=0.5572

🔍 Target: mu
  ➤ Tuning HGBR...
    ✓ HGBR: RMSE=1.0209, MAE=0.6889, R2=0.5616
  ➤ Tuning RF...
    ✓ RF: RMSE=1.0033, MAE=0.6745, R2=0.5765
  ➤ Tuning Ridge...
    ✓ Ridge: RMSE=1.2634, MAE=0.9197, R2=0.3285

🔍 Target: alpha
  ➤ Tuning HGBR...
    ✓ HGBR: RMSE=1.2854, MAE=0.7948, R2=0.9641
  ➤ Tuning RF...
    ✓ RF: RMSE=1.3165, MAE=0.7937, R2=0.9623
  ➤ Tuning Ridge...
    ✓ Ridge: RMSE=1.7777, MAE=1.2409, R2=0.9313

🔍 Target: homo
  ➤ Tuning HGBR...
    ✓ HGBR: RMSE=0.0105, MAE=0.0072, R2=0.8023
  ➤ Tuning RF...
    ✓ RF: RMSE=0.0101, MAE=0.0069, R2=0.8168
  ➤ Tuning Ridge...
    ✓ Ridge: RMSE=0.0149, MAE=0.0115, R2=0.6007

🔍 Target: lumo
  ➤ Tuning HGBR...
    ✓ HGBR: RMSE=0.0156, MAE=0.0108, R2=0.8816
  ➤ Tuning RF...
    ✓ RF: RMSE=0.0155, MAE=0.0104, R2=0.8835
  ➤ Tuning

In [ ]:
# --- Display benchmark results
print(benchmark_df)

                   RMSE        MAE        R2
Target Model                                
gap    HGBR    0.017862   0.012545  0.851947
       RF      0.017721   0.012163  0.854279
       Ridge   0.030892   0.025231  0.557163
mu     HGBR    1.020851   0.688871  0.561559
       RF      1.003336   0.674495  0.576474
       Ridge   1.263358   0.919680  0.328510
alpha  HGBR    1.285436   0.794800  0.964091
       RF      1.316487   0.793707  0.962336
       Ridge   1.777714   1.240922  0.931321
homo   HGBR    0.010473   0.007183  0.802304
       RF      0.010080   0.006911  0.816845
       Ridge   0.014884   0.011477  0.600663
lumo   HGBR    0.015604   0.010819  0.881562
       RF      0.015475   0.010392  0.883499
       Ridge   0.027082   0.021579  0.643218
r2     HGBR   74.426117  46.097928  0.906978
       RF     74.381408  45.331891  0.907089
       Ridge  93.348595  56.842963  0.853664
zpve   HGBR    0.004513   0.002892  0.979361
       RF      0.005024   0.002659  0.974420
       Rid